In [ ]:
!pip install -U transformers
!pip install -U  bitsandbytes
!pip install -U sentencepiece
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.9 MB/s eta 0:00:00


# This notebook must have GPU running kernel

In [ ]:
from transformers import  AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList
import torch

In [ ]:
class StopTokenCriteria(StoppingCriteria):
    def __init__(self, stop_tokens, tokenizer, prompt_length):
        self.stop_tokens = stop_tokens
        if tokenizer.pad_token not in stop_tokens:
            self.stop_tokens.append(tokenizer.pad_token)
        if tokenizer.bos_token not in stop_tokens:
            self.stop_tokens.append(tokenizer.bos_token)
        if tokenizer.eos_token not in stop_tokens:
            self.stop_tokens.append(tokenizer.eos_token)
        self.tokenizer = tokenizer
        self.prompt_length = prompt_length

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        is_done = False
        tokens = tokenizer.decode(input_ids[0])[self.prompt_length:]
        for st in self.stop_tokens:
            if st in tokens:
                is_done = True
                break
        return is_done

In [ ]:
model_name = "willnguyen/lacda-2-7B-chat-v0.1"
tokenizer = LlamaTokenizer.from_pretrained(
        model_name,
        use_fast=False,
        padding_side="right",
        tokenizer_type='llama',
)
tokenizer.pad_token_id = 0

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
     device_map="auto",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Please refer to this doc for more generation strategies detail
# https://huggingface.co/docs/transformers/main/en/generation_strategies
# Pass in a prompt and infer with the model
prompt = "<s> [INST] who is Hồ Chí Minh [/INST]"

stopping_criteria = StoppingCriteriaList([StopTokenCriteria(["[INST]", "[/INST]"], tokenizer, len(prompt))])
with torch.inference_mode():
    input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to('cuda')
    streamer = TextStreamer(tokenizer)
    _ = model.generate(
        input_ids=input_ids,
        max_new_tokens=1024,
        do_sample=False,
        temperature=1.0,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        use_cache=True,
        streamer=streamer,
        stopping_criteria=stopping_criteria
    )

<s> [INST] who is Hồ Chí Minh [/INST] Hồ Chí Minh was a Vietnamese revolutionary and politician who served as the first Prime Minister of the Democratic Republic of Vietnam (North Vietnam) from 1945 to 1969. [INST]
